In [2]:
from surprise import accuracy
from surprise import *
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
import pandas as pd
from surprise import NMF
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

In [3]:
# Load the data from a CSV file
df = pd.read_csv("selected_item_based.csv")
ratings = df.drop("Unnamed: 0", axis=1)
ratings.head()

,asin,overall,reviewerID
0,073530498X,4,A3VVMIMMTYQV5F
1,073530498X,5,A1XSPKZ8HHSBX2
2,073530498X,5,A2ZEQ0WBLNQN7O
3,073530498X,5,A2IC3RIPB6HKSQ
4,073530498X,5,A1JHNR71TPEETW


In [4]:
# Define a reader with the rating scale
reader = Reader(rating_scale=(1, 5))  # Assuming the ratings are from 1 to 5

# Load the data from the DataFrame into the surprise dataset
dataset = Dataset.load_from_df(ratings[['reviewerID', 'asin', 'overall']], reader)

model = SVD()

model.fit(dataset.build_full_trainset())

# import pickle

# with open('Model/svd_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [5]:
rr = ratings[ratings['reviewerID'] == "A1XSPKZ8HHSBX2"].sort_values(by="overall", ascending=False)
rr

,asin,overall,reviewerID
1,073530498X,5,A1XSPKZ8HHSBX2


In [10]:
items = ratings["asin"].unique()
test = [['A1XSPKZ8HHSBX2', iid, 3] for iid in items]
predictions = model.test(test)
pred = pd.DataFrame(predictions)
pred.head()

,uid,iid,r_ui,est,details
0,A1XSPKZ8HHSBX2,073530498X,3,4.551520,{'was_impossible': False}
1,A1XSPKZ8HHSBX2,1934148644,3,4.474340,{'was_impossible': False}
2,A1XSPKZ8HHSBX2,1934148423,3,4.019030,{'was_impossible': False}
3,A1XSPKZ8HHSBX2,1934148075,3,4.455455,{'was_impossible': False}
4,A1XSPKZ8HHSBX2,5135000011,3,3.729256,{'was_impossible': False}


In [11]:
def get_top_n(predictions, n=10):
    # Create a dictionary to store recommendations for each user
    top_n = defaultdict(list)

    # Group the predictions by user ID
    for user_id, item_id, true_rating, estimated_rating, _ in predictions:
        top_n[user_id].append((item_id, estimated_rating))

    # Sort the items for each user and retrieve the top-N recommendations
    for user_id, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[user_id] = user_ratings[:n]
    print()
    print("*"*100)
    return top_n

In [12]:
top_n = get_top_n(predictions, n=10)
top_n


****************************************************************************************************


defaultdict(list,
            {'A1XSPKZ8HHSBX2': [('B000053HH5', 5),
              ('B00009R90W', 5),
              ('B0000DYV9N', 5),
              ('B0002E3WIQ', 5),
              ('B0002I6Z72', 5),
              ('B000I1X3W8', 5),
              ('B000VR5YA8', 5),
              ('B000VRV6LY', 5),
              ('B0011NPHWQ', 5),
              ('B0015DPJ5W', 5)]})

In [13]:
user_id = "A1XSPKZ8HHSBX2"
recommendations = top_n[user_id]
print("Top 10 recommendations for user '{}':".format(user_id))
for item_id, rating in recommendations:
    print(f"- Item ID: {item_id}, Estimated Rating: {rating}")

Top 10 recommendations for user 'A1XSPKZ8HHSBX2':
- Item ID: B000053HH5, Estimated Rating: 5
- Item ID: B00009R90W, Estimated Rating: 5
- Item ID: B0000DYV9N, Estimated Rating: 5
- Item ID: B0002E3WIQ, Estimated Rating: 5
- Item ID: B0002I6Z72, Estimated Rating: 5
- Item ID: B000I1X3W8, Estimated Rating: 5
- Item ID: B000VR5YA8, Estimated Rating: 5
- Item ID: B000VRV6LY, Estimated Rating: 5
- Item ID: B0011NPHWQ, Estimated Rating: 5
- Item ID: B0015DPJ5W, Estimated Rating: 5
